## 1. Imports and Setup

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr
import os
from datetime import datetime


folder_name = '' ## Enter the name of the folder where you want to save the output files
# Output directory setup
def setup_output_folder():
    folder = folder_name
    os.makedirs(folder, exist_ok=True)
    return folder


## 2. Generate Datasets: Supplier X & Supplier Y

In [ ]:
def reverse_engineer_suppliers_from_company(company_data, gamma_x, gamma_y, seed=42):
    """
    Reverse-engineers Supplier X and Y lead times from fixed company data.
    Ensures that: X < Y < Company.
    """
    n = len(company_data)
    product_ids = company_data["ProductID"].values
    company_lead_times = company_data["Lead_Time"].values
    
    # Generate random production and internal delays
    np.random.seed(seed)
    supplier_y_max = []
    while len(supplier_y_max) < n:
        delay = np.random.gamma(shape = 2, scale = 1.5)
        if delay > 0 and delay <= company_lead_times[len(supplier_y_max)] - 3:
            supplier_y_max.append(company_lead_times[len(supplier_y_max)] - delay)

    # Randomly generate lead times for Supplier Y from a gamma distribution such that X < Y < Company 
    np.random.seed(seed + 2)
    supplier_y_lead = []
    while len(supplier_y_lead) < n:
        y_lead = np.random.gamma(shape = gamma_y[0], scale = gamma_y[1])
        if y_lead > 2 and y_lead < supplier_y_max[len(supplier_y_lead)]:
            supplier_y_lead.append(y_lead)

    # Randomly generate lead times for Supplier X from a gamma distribution such that X < Y < Company
    np.random.seed(seed + 3)
    supplier_x_lead = []
    while len(supplier_x_lead) < n:
        x_lead = np.random.gamma(shape = gamma_x[0], scale = gamma_x[1])
        if x_lead > 1 and x_lead < supplier_y_lead[len(supplier_x_lead)]:
            supplier_x_lead.append(x_lead)

    # Create DataFrames
    supplier_x_data = pd.DataFrame({'ProductID': product_ids, 'Lead_Time': supplier_x_lead})
    supplier_y_data = pd.DataFrame({'ProductID': product_ids, 'Lead_Time': supplier_y_lead})

    return supplier_x_data, supplier_y_data, company_data

## 3. Correlation Analysis Function

In [15]:
def analyze_lead_time_correlations(company_data, supplier_x_data, supplier_y_data, situation=None, output_dir="DP_Results", label="base", title_info=""):

    if situation is None: # base
        column_company = "Lead_Time"
        company_other = "Lead_Time"
    elif situation == "Situation1":
        column_company = "Noisy_Lead_Time"
        company_other = "Lead_Time"
    elif situation == "Situation2":
        column_company = "Noisy_Lead_Time"
        company_other = "Noisy_Lead_Time"
    else:
        raise ValueError("Invalid situation. Choose from: None, 'Situation1', 'Situation2'.")

    # column = "Noisy_Lead_Time" if noisy else "Lead_Time"

    min_length = min(len(company_data), len(supplier_x_data), len(supplier_y_data))
    company_data = company_data.iloc[:min_length].reset_index(drop=True)
    supplier_x_data = supplier_x_data.iloc[:min_length].reset_index(drop=True)
    supplier_y_data = supplier_y_data.iloc[:min_length].reset_index(drop=True)

    c = company_data[column_company]
    x = supplier_x_data[company_other]
    y = supplier_y_data[company_other]

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    sns.scatterplot(x=x, y=y, ax=axes[0], alpha=0.5)
    sns.scatterplot(x=y, y=c, ax=axes[1], alpha=0.5)
    sns.scatterplot(x=x, y=c, ax=axes[2], alpha=0.5)
    axes[0].set_title("Supplier X vs Supplier Y")
    axes[1].set_title("Supplier Y vs Company")
    axes[2].set_title("Supplier X vs Company")

    filename = f"{output_dir}/{title_info}_Correlations_Scatter_{label}.png"
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

    return {
        "Supplier X ↔ Supplier Y (Spearman)": spearmanr(x, y)[0],
        "Supplier Y ↔ Company (Spearman)": spearmanr(y, c)[0],
        "Supplier X ↔ Company (Spearman)": spearmanr(x, c)[0]
    }


## 4. Sensitivity Calculation

In [16]:
def calculate_sensitivities(company_df, output_dir="DP_Results"):
    mean_lead_time = company_df["Lead_Time"].mean()
    company_df["Deviation"] = abs(company_df["Lead_Time"] - mean_lead_time)
    max_deviation_row = company_df.loc[company_df["Deviation"].idxmax()]
    company_df_dprime = company_df.drop(index=max_deviation_row.name).drop(columns=["Deviation"])

    d = len(company_df)
    d_prime = len(company_df_dprime)
    delta_f_count = abs(d - d_prime)

    mean_d = company_df["Lead_Time"].mean()
    mean_dprime = company_df_dprime["Lead_Time"].mean()
    delta_f_mean = abs(mean_d - mean_dprime)

    company_df_dprime.to_csv(f"{output_dir}/Dataset_1c_Company_Dprime.csv", index=False)

    return delta_f_count, delta_f_mean, mean_d, mean_dprime


## 5. Apply Differential Privacy (Count and Mean)

In [17]:
def apply_dp_noise(df, sensitivity, epsilon, output_path):
    b = sensitivity / epsilon
    noise = np.random.laplace(loc=0, scale=b, size=len(df))
    noisy_df = df.copy()
    noisy_df["Noisy_Lead_Time"] = noisy_df["Lead_Time"] + noise
    noisy_df.to_csv(output_path, index=False)
    return noisy_df


## 6. MAE Calculation

In [18]:
def compute_mae_from_lists(baseline_list, private_list):
    return np.mean([abs(b - p) for b, p in zip(baseline_list, private_list)])


## 7. Histogram Error Analysis

In [20]:
def plot_histogram(df, title, x_label, filename, output_dir):
    plt.figure(figsize=(10, 6))
    plt.hist(df, bins=30, edgecolor='black', color='skyblue')
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{filename}.png")
    plt.close()


## 8. Main Function & Hyperparameter Tuning (Grid Search)

In [ ]:
def main():
    output_dir = setup_output_folder()

    epsilons = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5.0]
    sample_sizes = [7706]
    gamma_params = [
        {"x": (2, 3), "y": (2.5, 4)},

  

    ]
    situations = ["Situation1", "Situation2"]

    company_data_full = pd.read_csv("Company_Lead_Time_data.csv")

    for situation in situations:
        
        results = []

        for dist_id, dist in enumerate(gamma_params):
            for n in sample_sizes:
                for eps in epsilons:
                    title_info = f"{situation}/ε{eps}_n{n}_x{dist['x']}_y{dist['y']}".replace(" ", "")

                    # Step 1: Company sample
                    company_sample = company_data_full.sample(n=n, random_state=42).reset_index(drop=True)

                    # Step 2: Reverse engineer X and Y
                    sx, sy, co = reverse_engineer_suppliers_from_company(company_sample, dist["x"], dist["y"])

                    # Step 3: Save raw datasets
                    sx.to_csv(f"{output_dir}/{title_info}_Dataset_2_Supplier_X_with_dependencies.csv", index=False)
                    sy.to_csv(f"{output_dir}/{title_info}_Dataset_3_Supplier_Y_with_dependencies.csv", index=False)
                    co.to_csv(f"{output_dir}/{title_info}_Dataset_1_Company_with_dependencies.csv", index=False)

                    # Step 4: Plot histograms
                    plot_histogram(sx["Lead_Time"], "Supplier X Lead Time", "Lead Time", f"{title_info}_hist_supplierX", output_dir)
                    plot_histogram(sy["Lead_Time"], "Supplier Y Lead Time", "Lead Time", f"{title_info}_hist_supplierY", output_dir)
                    plot_histogram(co["Lead_Time"], "Company Lead Time", "Lead Time", f"{title_info}_hist_company", output_dir)

                    # Step 4: Baseline correlations
                    base_corrs = analyze_lead_time_correlations(co, sx, sy, output_dir=output_dir, label="baseline", title_info=title_info)
                    
                    # Step 5: Sensitivities
                    delta_count, delta_mean, _, _ = calculate_sensitivities(co, output_dir)

                    # Step 6: Apply DP to all or the company only (depending on the situation)
                    if situation == "Situation1": # Situation 1
                        d4_co_mean = apply_dp_noise(co, delta_mean, eps, f"{output_dir}/{title_info}_Dataset_4_Company_DP_MeanLeadTimeQuery.csv")
                        d4_co_count = apply_dp_noise(co, delta_count, eps, f"{output_dir}/{title_info}_Dataset_4_Company_DP_CountQuery.csv")
                      
                        # Step 7: Correlations with DP data
                        dp_corr_mean = analyze_lead_time_correlations(d4_co_mean, sx, sy, situation="Situation1", output_dir=output_dir, label="dp_mean", title_info=title_info)

                    else: # Situation 2
                        d4_co_mean = apply_dp_noise(co, delta_mean, eps, f"{output_dir}/{title_info}_Dataset_4_Company_DP_MeanLeadTimeQuery.csv")
                        d4_co_count = apply_dp_noise(co, delta_count, eps, f"{output_dir}/{title_info}_Dataset_4_Company_DP_CountQuery.csv")
                        d4_sx = apply_dp_noise(sx, delta_mean, eps, f"{output_dir}/{title_info}_Dataset_4_Supplier_X_DP.csv")
                        d4_sy = apply_dp_noise(sy, delta_mean, eps, f"{output_dir}/{title_info}_Dataset_4_Supplier_Y_DP.csv")

                        # Step 7: Correlations with DP data
                        dp_corr_mean = analyze_lead_time_correlations(d4_co_mean, d4_sx, d4_sy, situation="Situation2", output_dir=output_dir, label="dp_mean", title_info=title_info)

                    # Step 8: MAE per relationship
                    mae_x_y = abs(base_corrs["Supplier X ↔ Supplier Y (Spearman)"] - dp_corr_mean["Supplier X ↔ Supplier Y (Spearman)"])
                    mae_y_c = abs(base_corrs["Supplier Y ↔ Company (Spearman)"] - dp_corr_mean["Supplier Y ↔ Company (Spearman)"])
                    mae_x_c = abs(base_corrs["Supplier X ↔ Company (Spearman)"] - dp_corr_mean["Supplier X ↔ Company (Spearman)"])

                    # Step 9: Save histograms
                    # plot_error_histogram(co, d4_co_mean, "Error - Company Mean Query", f"{title_info}_hist_mean_company", output_dir)
                    plot_histogram(d4_co_mean["Noisy_Lead_Time"] - co["Lead_Time"], "Company Lead Time with DP", "Lead Time Error (Noisy - True)", f"{title_info}_hist_dp_company", output_dir)

                    if situation == "Situation 2":
                        plot_histogram(d4_sx["Noisy_Lead_Time"] - sx["Lead_Time"], "Supplier X Lead Time with DP", "Lead Time Error (Noisy - True)", f"{title_info}_hist_dp_supplierX", output_dir)
                        plot_histogram(d4_sy["Noisy_Lead_Time"] - sy["Lead_Time"], "Supplier Y Lead Time with DP", "Lead Time Error (Noisy - True)", f"{title_info}_hist_dp_supplierY", output_dir)

                        # plot_error_histogram(sx, d4_sx, "Error - Supplier X", f"{title_info}_hist_dp_supplierX", output_dir)
                        # plot_error_histogram(sy, d4_sy, "Error - Supplier Y", f"{title_info}_hist_dp_supplierY", output_dir)

                    # Step 10: Store result
                    results.append({
                        "epsilon": eps,
                        "sample_size": n,
                        "gamma_x": dist["x"],
                        "gamma_y": dist["y"],
                        "MAE_XY": mae_x_y,
                        "MAE_YC": mae_y_c,
                        "MAE_XC": mae_x_c,
                    })

        # Step 11: Save results
        results_df = pd.DataFrame(results)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_path = f"{output_dir}/{situation}/Grid_Search_Results_{timestamp}.xlsx"
        results_df.to_excel(results_path, index=False)
        print(f" Grid search completed. Results saved to {results_path}")

        # Step 12: Plot efficient frontiers for each correlation
        relationships = {
            "MAE_XY": "Supplier X ↔ Supplier Y",
            "MAE_YC": "Supplier Y ↔ Company",
            "MAE_XC": "Supplier X ↔ Company"
        }

        for mae_key, label in relationships.items():
            plt.figure(figsize=(10, 6))
            for n in sample_sizes:
                subset = results_df[results_df["sample_size"] == n]
                plt.plot(subset[mae_key], subset["epsilon"], marker='o', label=f"Sample Size: {n}")
            plt.gca().invert_xaxis()
            plt.gca().invert_yaxis()
            plt.xlabel("Accuracy (Lower MAE → Better)")
            plt.ylabel("Privacy (Lower Epsilon → Better)")
            plt.title(f"Efficient Frontier: {label} in {situation}")
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(f"{output_dir}/{situation}/Efficient_Frontier_{mae_key}.png")
            plt.show()




# Run the main function
main()
